In [42]:
import h5py
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2 as cv
#print(os.listdir('./data/'))
noExamples = len(os.listdir('./data'))
data = np.zeros((noExamples,224,224,1))
labels = np.zeros((noExamples,1))
for itemId, file in zip(range(0,noExamples),os.listdir('./data')):
    with h5py.File('./data/'+file, 'r') as f:
        tmpdata = np.array(f['cjdata']['image'])
        data[itemId,:,:,:] = cv.resize(tmpdata, (224,224)).reshape(224,224,1)
        labels[itemId] = f['cjdata']['label']



In [43]:
x_train,x_test = data[0:700,:,:,:]/255,data[700:,:,:,:]/255
y_train,y_test = labels[0:700], labels[700:]


In [ ]:
 model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=48,kernel_size=4,padding="valid", activation='relu', input_shape=(224,224,1)), 
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax),
  
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
print()
model.evaluate(x_test, y_test)

#print(data[0,200:210,200:210])
#plt.imshow(data[0,:,:])
#plt.show()
#print(data.shape)
#print(labels.shape)
#print(labels)


Epoch 1/5
700/700 [==============================] - 18s 26ms/sample - loss: 16.4447 - accuracy: 0.9029
Epoch 2/5
700/700 [==============================] - 2496s 4s/sample - loss: 1.4612 - accuracy: 0.9757
Epoch 3/5
512/700 [====================>.........] - ETA: 4s - loss: 0.0320 - accuracy: 0.9980